In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from prep import get_data
from torch import nn
from kfold_experiments import kfold_normal_training_softgedt, kfold_adversarial_training_softgedt,kfold_normal_training_neural_networks, kfold_adversarial_training_neural_networks

# Create Dataset

In [2]:
# load dataset
filename = 'vowel'
data, ncat,dtypes = get_data(filename)  # Preprocess the data

# Preprocess the data without standardizing
data, ncat,dtypes = get_data(filename)

# create editability vector
non_editable_vector = torch.tensor([1 for i in range(data[:,:-1].shape[1])])
non_editable_vector[0] = 0
non_editable_vector[1] = 0

# record datatypes
dtypes[dtypes=='object'] = 'pos_int'
dtypes[dtypes=='float64'] = 'float'

datatypes = dtypes[:-1].to_numpy().astype('object')
datatypes[(datatypes=='int64') & np.min(data[:,:-1]>=0,axis=0)]='pos_int'
datatypes[(datatypes=='float64') & np.min(data[:,:-1]>=0,axis=0)]='pos_float'
datatypes = datatypes.tolist()

# Normal Training

In [3]:
kfold_normal_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight = True)

experiment: 0
Fold: 0
epoch: 0
train acc: tensor(0.9377) valid acc: tensor(0.9040) trigger times: 0
epoch: 1
train acc: tensor(0.9512) valid acc: tensor(0.8737) trigger times: 0
epoch: 2
train acc: tensor(0.9512) valid acc: tensor(0.8838) trigger times: 1
epoch: 3
train acc: tensor(0.9545) valid acc: tensor(0.8990) trigger times: 0
epoch: 4
train acc: tensor(0.9579) valid acc: tensor(0.8788) trigger times: 0
epoch: 5
train acc: tensor(0.9596) valid acc: tensor(0.8838) trigger times: 1
epoch: 6
train acc: tensor(0.9529) valid acc: tensor(0.8838) trigger times: 0
epoch: 7
train acc: tensor(0.9545) valid acc: tensor(0.8838) trigger times: 1
epoch: 8
train acc: tensor(0.9596) valid acc: tensor(0.8838) trigger times: 2
epoch: 9
train acc: tensor(0.9596) valid acc: tensor(0.8889) trigger times: 3
Low density samples found at indices  tensor([105]) . Probably out-of-domain samples.
SoftGeDT Accuracy on Training set: tensor(0.9596)
SoftGeDT Accuracy on Validation set: tensor(0.8889)
SoftGeDT A

# Adversarial Training

In [8]:
kfold_adversarial_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight=True)

Fold: 0
epoch: 0
train error: 0.19865319865319866 train loss: 1.8896201547950204 validation error 0.41414141414141414
epoch: 1
train error: 0.16666666666666666 train loss: 0.8576595839426574 validation error 0.3838383838383838
epoch: 2
train error: 0.16161616161616163 train loss: 0.6806880584870926 validation error 0.3838383838383838
epoch: 3
train error: 0.15151515151515152 train loss: 0.5945675284774216 validation error 0.3888888888888889
epoch: 4
train error: 0.1447811447811448 train loss: 0.5390255057851875 validation error 0.37373737373737376
epoch: 5
train error: 0.132996632996633 train loss: 0.5282375740282463 validation error 0.36363636363636365
epoch: 6
train error: 0.13468013468013468 train loss: 0.5434242868262911 validation error 0.37373737373737376
epoch: 7
train error: 0.12794612794612795 train loss: 0.5336617267493046 validation error 0.37373737373737376
epoch: 8
train error: 0.12794612794612795 train loss: 0.5096919392094468 validation error 0.37373737373737376
epoch: 9

# Simple Neural Network

In [3]:
# Define structure
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim, 80).double()
        self.layer2 = nn.Linear(80, 80).double()
        self.layer3 = nn.Linear(80, 11).double()
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

## 5-fold cross validation normal training

In [4]:
kfold_normal_training_neural_networks(nn, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train acc: tensor(0.3047) valid acc: tensor(0.3990) trigger times: 0
epoch: 1
train acc: tensor(0.5741) valid acc: tensor(0.7172) trigger times: 0
epoch: 2
train acc: tensor(0.6886) valid acc: tensor(0.7424) trigger times: 0
epoch: 3
train acc: tensor(0.7795) valid acc: tensor(0.6818) trigger times: 0
epoch: 4
train acc: tensor(0.8165) valid acc: tensor(0.8030) trigger times: 1
epoch: 5
train acc: tensor(0.8670) valid acc: tensor(0.7576) trigger times: 0
epoch: 6
train acc: tensor(0.8923) valid acc: tensor(0.8434) trigger times: 1
epoch: 7
train acc: tensor(0.9209) valid acc: tensor(0.8636) trigger times: 0
epoch: 8
train acc: tensor(0.9562) valid acc: tensor(0.8737) trigger times: 0
epoch: 9
train acc: tensor(0.9545) valid acc: tensor(0.9192) trigger times: 0
epoch: 10
train acc: tensor(0.9512) valid acc: tensor(0.9444) trigger times: 0
epoch: 11
train acc: tensor(0.9731) valid acc: tensor(0.9091) trigger times: 0
epoch: 12
train acc: tensor(0.9663) valid acc: tensor(

## 5 fold cross validation adversarial training

In [26]:
kfold_adversarial_training_neural_networks(Net, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train error: 0.7676767676767676 train loss: 2.180511226174444 validation error 0.6818181818181818
epoch: 1
train error: 0.5740740740740741 train loss: 1.3995915858881571 validation error 0.6515151515151515
epoch: 2
train error: 0.48653198653198654 train loss: 1.1479259138298434 validation error 0.5151515151515151
epoch: 3
train error: 0.4074074074074074 train loss: 0.9630440336663686 validation error 0.4494949494949495
epoch: 4
train error: 0.3333333333333333 train loss: 0.8082385589615942 validation error 0.48484848484848486
epoch: 5
train error: 0.2996632996632997 train loss: 0.7158096588503968 validation error 0.41919191919191917
epoch: 6
train error: 0.27946127946127947 train loss: 0.6332804335762543 validation error 0.36363636363636365
epoch: 7
train error: 0.25757575757575757 train loss: 0.6270754702142719 validation error 0.42424242424242425
epoch: 8
train error: 0.265993265993266 train loss: 0.6219132921889733 validation error 0.41919191919191917
epoch: 9
train